<a href="https://colab.research.google.com/github/Maks2811/test_nagaev/blob/main/Task_14_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date, col, sum, count

spark = SparkSession.builder.appName("WebLogAnalyzer").getOrCreate()

# создаем датафрейм из считываемого csv файла
df_web_log = spark.read.csv("/content/web_server_logs.csv", header=True, inferSchema=True)

# выводим схему и верхние 20 строк датафрейма
df_web_log.printSchema()
df_web_log.show()

# 1. выводим 10 самых активных IP
print('Top 10 active IP addresses:')

top_ip = df_web_log.groupBy("ip").agg(count("ip").alias("request_count"))
top_ip.orderBy(col("request_count").desc()).limit(10).show()

# 2. группируем данные по HTTP-методу и считаем количество запросов для каждого метода
print('Request count by HTTP method:')

meth_count = df_web_log.groupBy("method").agg(count("method").alias("method_count"))
meth_count.show()

# 3. считаем количество запросов с кодом ответа 404.

count_404 = df_web_log.filter(col("response_code") == 404).count()
print(f'Number of 404 response codes: {count_404}')

# Проверка количества через группировку с агрегацией
# df_web_log.filter(col("response_code") == 404).groupBy("response_code").agg(count("method")).show()

# 4. группируем данные по дате, суммируем размер ответов, сортируем по дате.

print('\nTotal response size by day:')

# выделяем из timestamp дату и записываем в отдельную колонку
df_web_log_date = df_web_log.withColumn("date", to_date(df_web_log["timestamp"], "yyyy-MM-dd"))

# группируем по дате, суммируем размеры ответов, сортируем
total_response = df_web_log_date.groupBy("date").agg(sum("response_size").alias("total_response_size")).orderBy(col("date"))
total_response.show()

root
 |-- ip: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- method: string (nullable = true)
 |-- url: string (nullable = true)
 |-- response_code: integer (nullable = true)
 |-- response_size: integer (nullable = true)

+---------------+--------------------+------+--------------------+-------------+-------------+
|             ip|           timestamp|method|                 url|response_code|response_size|
+---------------+--------------------+------+--------------------+-------------+-------------+
|108.215.184.121|2024-05-11 09:07:...|   GET|explore/blog/cate...|          404|         4586|
| 111.201.198.91|2024-06-13 09:34:...|DELETE| category/categories|          200|          695|
|  98.46.195.117|2024-07-02 09:15:...|   GET|      app/categories|          500|         8705|
|  76.18.169.221|2024-07-24 10:44:...|   PUT|              search|          200|         7675|
|  33.19.155.166|2024-02-21 15:43:...|   PUT|     tags/wp-content|          301|      